# Unpacking

- `*` operator captures excess items during unpacking.

In [3]:
numbers = [1,2,3,4,5]
first, *middle, last = numbers
print(first)
print(middle)
print(last)

1
[2, 3, 4]
5


# Functions

## Variable-Length Positional Arguments
- `*args` convention

In [12]:
def sum_all(*subrata):
    print(sum(subrata))
    print(type(subrata), subrata)

sum_all(1,2,3,4,5,6,7,8,9)

45
<class 'tuple'> (1, 2, 3, 4, 5, 6, 7, 8, 9)


## Variable-Length Keyword Arguments

- `*kwargs` convention

In [13]:
def info(**subrata):
    print(subrata.items())
    print(type(subrata), subrata)

info(firstname="Subrata", lastname="Mondal")

dict_items([('firstname', 'Subrata'), ('lastname', 'Mondal')])
<class 'dict'> {'firstname': 'Subrata', 'lastname': 'Mondal'}


## Lambda Functions

In [17]:
add = lambda x, y : x + y
add(5, 6)

11

In [22]:
numbers = [1,2,3,4,5,6,7,8,9]
squared = list(map(lambda x:x*x, numbers))
print(squared)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


## Decorators
- **Higher Order Function** that modifies the behaviour of another function. It is used for **logging, caching, access control** without modifying the original function's code.

> **In LangGraph, Decorators can be used to wrap Node functions, adding Logging or Error-Handling behaviours.**

In [25]:
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Executing: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@log_decorator
def greet(name):
    print(f"Hello, {name}!")
    
greet("Alice")

Executing: greet
Hello, Alice!


In [26]:
import time

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)  # Call the original function
        print(f"{func.__name__} took {time.time() - start:.2f}s")
        return result
    return wrapper

@timer_decorator
def slow_function():
    time.sleep(2)

slow_function()  # Output: "slow_function took 2.00s"

slow_function took 2.00s


In [28]:
def log_arguments_and_result(func):
    def wrapper(length, width, height):  # Explicit parameters for clarity
        print(f"Calling {func.__name__} with args: {length}, {width}, {height}")
        result = func(length, width, height)  # Call the original function
        print(f"Result: {result}")
        return result
    return wrapper

@log_arguments_and_result
def calculate_volume(length, width, height):
    return length * width * height

calculate_volume(3, 4, 5)

Calling calculate_volume with args: 3, 4, 5
Result: 60


60

In [29]:
def log_arguments_and_result(func):
    def wrapper(*args, **kwargs):  # Accept any arguments
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)  # Pass all arguments to the original function
        print(f"Result: {result}")
        return result
    return wrapper

@log_arguments_and_result
def calculate_volume(length, width, height=1):  # Now with a default parameter
    return length * width * height

calculate_volume(3, 4, height=5)  # Mix positional and keyword arguments

Calling calculate_volume with args: (3, 4), kwargs: {'height': 5}
Result: 60


60

## **Closures Explained**
A **closure** is a function that retains access to variables from its enclosing **(outer) scope**, even after the outer function has finished executing. In decorators, the `wrapper` function is a closure because it remembers the `func` parameter from the outer `log_arguments_and_result` function.

**Example of a Closure**
```python
def outer_function(message):
    def inner_function():
        print(message)  # inner_function "closes over" the 'message' variable
    return inner_function

my_closure = outer_function("Hello, closure!")
my_closure()  # Output: "Hello, closure!"
```

- **Key Point**: Even though `outer_function` has finished executing, `inner_function` still has access to `message`.

**Closures in Decorators**
In the decorator example:
- `log_arguments_and_result` (outer function) takes `func` as a parameter.
- The `wrapper` (inner function) uses `func` even after `log_arguments_and_result` has returned.
- This is only possible because `wrapper` is a closure that "closes over" `func`.

---

### **Why Use Closures in Decorators?**
1. **Encapsulation**: The decorator’s logic (e.g., logging) is neatly wrapped around the original function.
2. **State Retention**: The closure retains access to the original function (`func`) and any variables from the decorator’s scope.

---

### **Advanced Example: Parameter Validation Decorator**
Let’s create a decorator that validates if all parameters are positive:

```python
def validate_positive_numbers(func):
    def wrapper(*args, **kwargs):
        # Check all positional and keyword arguments
        for arg in args:
            if not isinstance(arg, (int, float)) or arg <= 0:
                raise ValueError("All arguments must be positive numbers")
        for value in kwargs.values():
            if not isinstance(value, (int, float)) or value <= 0:
                raise ValueError("All arguments must be positive numbers")
        return func(*args, **kwargs)
    return wrapper

@validate_positive_numbers
def calculate_area(length, width):
    return length * width

print(calculate_area(4, 5))  # Works: 20
calculate_area(-2, 5)        # Raises ValueError
```

In [31]:
# Timing + Logging Decorator
import time
from functools import wraps  # To preserve function metadata

def timer_and_logger(func):
    @wraps(func)  # Preserves func's name/docstring
    def wrapper(*args, **kwargs):
        print(f"Starting {func.__name__} with args: {args}, kwargs: {kwargs}")
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Finished {func.__name__} in {end_time - start_time:.2f} seconds")
        return result
    return wrapper

@timer_and_logger
def complex_operation(a, b, c=1):
    """Multiplies three numbers after a delay."""
    time.sleep(1)
    return a * b * c

print(complex_operation(2, 3, c=4))  # Output: 24 (with logs and timing)

Starting complex_operation with args: (2, 3), kwargs: {'c': 4}
Finished complex_operation in 1.01 seconds
24


---